# Finance Benchmark

In [ ]:
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import itertools as it
import matplotlib.pyplot as plt
import json

As explained in notebook: **01_Benchamark_Finanzas** the **PriceEstimation** class from **finance_benchmark.py** module solves a complete estimation price problem. The input of the class is large python dictionary where the estimation problem and the solver is configured.

For automatize the execution of different estimation problems and different solvers the module **dictionary_combination.py** was created.

Following sections explains how to use the functions of the module.

## 1. Definitions.

We are going to use the following definitions:

* **Price Problem** (**PriceP** from now): this will be a complete and properly configured price estimation of a pay off, under a probability density for a domain. It will be a python dictionary. 

Following cell we give an example of a  **PriceP** dictionary,


In [ ]:
price_p = {
    #Pay Off Configuration
    'pay_off_type': 'European_Put_Option',
    'strike': 1.5,
    'coupon': None,
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    #Domain Configuration
    'x0': 0.01,
    'xf': 3.0,
    'n_qbits': 7,
}

Other important definition will be 
* **Amplitude Estimation Price Problem** (**AE_PriceP** from now): this will be a price problem (**PriceP**)  with a properly configurated amplitude estimation method for solving the problem. It will be a python dictionary.

This is a complete dictionary for given to the **Price Estimation** class explained in the notebook: **01_Benchamark_Finanzas**.

Following cell shows an example of a **AE_PriceP**

In [ ]:
#Example of complete dictionary
ae_price_p = {
    #Amplitude Estimation selection
    'ae_type': 'MLAE',
    #Amplitude Estimation configuration
    'schedule': None,
    'mcz_qlm': False,
    'delta' : 1.0e-6,
    'ns' : 10000,
    'auxiliar_qbits_number': None,
    'cbits_number': None,
    'alpha': None,
    'gamma': None,
    'epsilon': None,
    'shots': None,
    'probability_loading': False,
    #Numbe of problem to solve
    'number_of_tests': 1,
    #PayOff Configuration
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,
    #Domain configuration
    'x0': 0.01,
    'xf': 3.5,
    'n_qbits': 5,
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5
}

## 2 Creating Amplitude Estimation Price Problem.

The **dictionary_combination.py** have programmed several functions that allow the use to create in an easy way diferent **AE_PriceP** (this is list of python dictionaries for given to the **Price Estimation** class. 

The functions programmed for this will be:

1. combination_for_dictionary
2. combination_for_list
3. create_pep_list
4. create_pricep_list_from_jsons
5. create_ae_pricep_list

### 2.1 combination_for_dictionary function

The input of this function is a python dictionary where for each key the values **MUST BE**  a list.

For a given key the correspondient list can be one ore more elements. For each element of the list a different output dictionary will be generated. If different keys have list with more than one element all posible combinations will be returned by the function.

In the following cells we give an example of how this work:

In [ ]:
from dictionary_combination import combination_for_dictionary

We are going to start with a *probability_dictionary* configuration where we have following list with more than one element:

* *risk_free_rate*  with 2 posible values
* *volatility*: with 3 posible values.

The **combination_for_dictionary** will create a list of python dictionaries wwhere all the posibilities will be exhausted: in this case we will have *risk_free_rate*volatility* number of dictionaries: 2*3=6

In [ ]:
probability_dictionary = {
    'probability_type': ['Black-Scholes'],
    's_0': [1],
    'risk_free_rate': [0.02, 0.05],
    'maturity': [1.0],
    'volatility': [0.1, 0.3, 0.5]
}
list_of_probabilities = combination_for_dictionary(probability_dictionary)
print(len(list_of_probabilities))

In [ ]:
list_of_probabilities[0]

### 2.2 combination_for_list function

The input of this function is a list of python dictionaries. Each python dictionary have a key value pair where the value is a python list. This function iterate over each dictionary of the input list and get all the posible combinations of the dictionary (using the *combination_for_dictionary* function) and finally concatenates all the lists.

Following cells show how this function works.

In [ ]:
from dictionary_combination import combination_for_list

In this case we have 2 different payoffs (an *European_Put_Option* and a *Futures*) and we want to test several *strikes*. So the *combination_for_list* will create all the posible configuration for each dictionary of the list and concatenate them.

So we will have 2 posible dictionaries from the *European_Put_Option* one and 3 combinations from the *futures* one. So our final list will have 5 posible dictionaries.

In [ ]:
payoffs = [
    {
        'pay_off_type': ['European_Put_Option'],
        'strike': [1.5, 2.0],
        'coupon': [None]
    },
    {
        'pay_off_type': ['Futures'],
        'strike': [0.5, 1.0, 1.5], 
        'coupon': [None]}
]
list_for_payoffs = combination_for_list(payoffs)
len(list_for_payoffs)

In [ ]:
list_for_payoffs[4]

### 2.3. create_pricep_list

This function will create a list of different **PriceP**. The function creates a list of dictionaries where each one is a **PriceP**. The inputs of the function will be:

* probability_list: list with different dictionaries for configure a density destribution
* payoff_list: list with different dictionaries for configure a pay off function
* domain_list: list with different dictionaries for configure a domain.

The function exhaust all the dictionaries for each input (using *combination_for_list*) and then create all the posible combinations of each input, taking into account tha for define a *PriceP* we need:
1. Probability density
2. Payoff
3. Domain

So the output list will be the combination of the all posible density, payoff and domains configurations.

Following cells show how this work

In [ ]:
from dictionary_combination import create_pricep_list

In [ ]:
#Probability
prob_list = [{
    'probability_type': ['Black-Scholes'],
    's_0': [1],
    'risk_free_rate': [0.05, 0.01],
    'maturity': [1.0],
    'volatility': [0.1, 0.3, 0.5]
}]

#Pay off
po_list =[
    {'pay_off_type': ['European_Call_Option'],
    'strike': [0.5],
    'coupon': [None]},
     {'pay_off_type': ['Futures'],
      'strike': [0.5, 1.0, 1.5],
      'coupon': [None]
     }
]

#Domain
do_list = [{
    'x0': [0.01], 'xf': [3.0], 'n_qbits': [7]
}]

In [ ]:
#From the prob_list we will have following elements
print('List for probability list:', len(combination_for_list(prob_list)))
#From the po_list we will have following elements
print('List for payoffs list:', len(combination_for_list(po_list)))
#From the do_list we will have following elements
print('List for domain list:', len(combination_for_list(do_list)))

The **create_pep_list** will create all the posible combinations from *probability list*, *payoffs list* and *domain list*. So the final list will have 6*4*1: 24 elements

In [ ]:
price_p = create_pricep_list(prob_list, po_list, do_list)

In [ ]:
len(price_p)

In [ ]:
price_p[16]

### 2.4. create_pricep_list_from_jsons

This function creates the **PriceP** list using json inputs

In [ ]:
from dictionary_combination import create_pricep_list_from_jsons

In [ ]:
problem_list = create_pricep_list_from_jsons(
    json_density="jsons/test_density_probability.json",
    json_payoffs="jsons/test_payoffs.json",
    json_domain="jsons/test_domain_configuration.json"
)

In [ ]:
len(problem_list)

In [ ]:
problem_list[0]

### 2.5 create_ae_pricep_list

This function creates a list of *AE_PriceP*. Each element will be a dictionary that can be given as input to the **PriceEstimation** class. The inputs are:

* ae_list: list of dictionaries with the properly configuration of amplitude estimation method.
* problem_list: list with *PriceP* problems. In general should be a list from *create_pricep_list* or *create_pricep_list_from_jsons* functions.

The result will be all possible combinations of an amplitude estimation method and a *PriceP* problem.


In [ ]:
from dictionary_combination import create_ae_pricep_list

Folowing cell shows a posible creation workflow of a **ae_list**. In this case qe are going to use two different *amplitude estimation* methods (**MLAE** y **IQAE**). Additionally for the **MLAE** one we are going to use 2 different *delta*. Using *combination_for_list* function we can exhasut all *amplitude amplification* configurations (3 in total)

In [ ]:
mlae_list = [
    {
        'ae_type': ['MLAE'],
        "mcz_qlm": [False],
        'probability_loading': [False],
        'schedule': [[[1, 100, 110, 120, 125, 130, 135, 140, 145, 150, 155],
        [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]]],
        'delta': [1e-09, 1e-06],
        'auxiliar_qbits_number': [None],
        'cbits_number': [None],
        'alpha': [None],
        'gamma': [None],
        'epsilon': [None],
        'shots': [None],
        'number_of_tests': [1]
    },
    {
        'ae_type': ['IQAE'],
        "mcz_qlm": [False],
        'probability_loading': [True],
        'schedule': [None],
        'delta': [None],
        'auxiliar_qbits_number': [None],
        'cbits_number': [None],
        'alpha': [0.05],
        'gamma': [None],
        'epsilon': [0.0001],
        'shots': [100],
        'number_of_tests': [1]
    }
]

In [ ]:
ae_list = combination_for_list(mlae_list)
len(ae_list)

In [ ]:
problem_list = create_pricep_list_from_jsons(
    json_density="jsons/test_density_probability.json",
    json_payoffs="jsons/test_payoffs.json",
    json_domain="jsons/test_domain_configuration.json"
)
len(problem_list)

In this case we are going to have 3 amplitude estimation methods that will be applied over 24 **PriceP** problems. So we will have 72 **AE_priceP** problems

In [ ]:
final_list = create_ae_pricep_list(ae_list, problem_list)
len(final_list)

Each element of final_list is a complete **AE_priceP** dictionaqry that can be given to the **EstimationPrice** class

In [ ]:
final_list[0]

Now we can combine with a **PriceP** list:

### 2.6 create_ae_pricep_list_from_jsons

This function creates the **AE_priceP** dictionary list from input jsons. 
In the *jsons/ae_test_conf.json* we have the json file with the same information that *mlae_list* from section 2.5. We are going to use for creating the complete list of **AE_PriceP** dictionaries.

In [ ]:
from dictionary_combination import create_ae_pricep_list_from_jsons

In [ ]:
with open("jsons/ae_test_conf.json") as json_file:
    ae_test_conf = json.load(json_file)
ae_list = combination_for_list(ae_test_conf)

print(len(ae_list))

In [ ]:
problem_list = create_pricep_list_from_jsons(
    json_density="jsons/test_density_probability.json",
    json_payoffs="jsons/test_payoffs.json",
    json_domain="jsons/test_domain_configuration.json"
)
len(problem_list)

This json will generate 3 **amplitude estimation** configurations. And they will be combine with the 24 *PriceP* configuration of the problem_list from section 2.4. 

So a 72 list of **AE_PriceP** dictionaries should be created!!

In [ ]:
other_jsons = {
    "json_density": "jsons/test_density_probability.json",
    "json_payoffs": "jsons/test_payoffs.json",
    "json_domain": "jsons/test_domain_configuration.json"
}
ae_pricep =create_ae_pricep_list_from_jsons(
    ["jsons/ae_test_conf.json"],
    **other_jsons

)

In [ ]:
len(ae_pricep)

In [ ]:
ae_pricep[0]

## 3. Configuration jsons

The configuration jsons files are the most important part for using the **dictionary_combination.py**. They are loacated in the jsons folder. We are going to divide this json in 2 types:

1. Problem configuration jsons:
    * domain_configuration.json : a json with the complete information for creating the domain (the x) of our price estimation problem.
    * density_probability.json : this json allows to configurate the probability density of the price estimation problem.
    * payoffs.json: in this json different payoffs and corresponding configurations are setting.
2. Amplitude Estimation jsons: 
    * mlae_configuration.json: for configuring **MLAE** class.
    * iqae_configuration.json: for configuring **IQAE** class.
    * rqae_configuration.json: for configuring **RQAE** class.
    * cqpeae_configuration.json: for configuring **CQPEAE** class.
    * iqpeae_configuration.json: for configurin **IQPEAE** class
    

## 5. run functions

We have created 2 *run* functions:

* **run_id**: for solving only one complete estimation price problem
* **run_staff**: for solving several complete estimation price problems

### 5.1. run_id

This function solves an input complete estimation problem. Inputs are:

* pe_problem: dictionary with all the complete information for a price estimation problem including amplitude estimation solver and configuration
* id_name: name for giving to the estimation problem for saving purpouses
* folder_name:  folder name for saving the results of the solution of the price estimation problem
* qlmaas: For usign a QLM as a Service for solving the price estimation problem
* save: For saving the results of the the price estimation problem as a csv

The return will be the an object of the **PriceEstimation** class where the solver was executed.

In [ ]:
from dictionary_combination import run_id

In [ ]:
opa = run_id(final[0], 'None', save=False)

In [ ]:
opa.pc.probability(opa.domain)

In [ ]:
opa.pay_off

In [ ]:
opa.pc.density_probability(0.7)

In [ ]:
plt.plot(np.linspace(opa.x0, opa.xf, 1000), opa.pc.density_probability(np.linspace(opa.x0, opa.xf, 1000)))

In [ ]:
%matplotlib inline

In [ ]:
opa.pc.density_probability(opa.domain)

In [ ]:
opa.domain

In [ ]:
opa.pc.density_probability(np.linspace(opa.x0, opa.xf, 10)).sum()/10

In [ ]:
opa.pc.density_probability(np.linspace(opa.x0, opa.xf, 100)).sum()

In [ ]:
from scipy import integrate

In [ ]:
integrate.quad(opa.pc.density_probability, opa.x0, opa.xf)

In [ ]:
suma = [] 
for i in range(1000):
    step = (opa.xf-opa.x0)*np.random.random_sample() + opa.x0
    suma.append(
        opa.pc.density_probability(
            step
        )*opa.po.pay_off(step)
    )
np.sum(suma)/len(suma)

In [ ]:
step = (opa.xf-opa.x0)*np.random.random_sample(100000) + opa.x0
final = opa.pc.density_probability(step)*opa.po.pay_off(step)
np.sum(final)/len(final)

In [ ]:
np.sum(suma)/len(suma)

In [ ]:
np.sum(
    opa.probability*opa.pay_off_normalised

)

In [ ]:
exacta = integrate.quad(a, opa.x0, opa.xf)[0]

In [ ]:
a(0.2)

In [ ]:
n = 100000
a = lambda x: opa.pc.density_probability(x)*opa.po.pay_off(x)
vals = np.random.uniform(opa.x0, opa.xf, n)
y = [a(val) for val in vals]
y_mean = np.sum(y)/n
integ = (opa.xf-opa.x0) * y_mean
print(integ-exacta)

In [ ]:
1.0/np.sqrt(n)